# Goal: Investigating a Drop in User Engagement

__About Yammer__: Yammer is a social network for communicating with coworkers. Individuals share documents, updates, and ideas by posting them in groups. Yammer is free to use indefinitely, but companies must pay license fees if they want access to administrative controls, including integration with user management systems like ActiveDirectory

__Problem__: You show up to work Tuesday morning, September 2, 2014. The head of the Product team
walks over to your desk and asks you what you think about the latest activity on the user
engagement dashboards. 
The chart shows the number of engaged users each week. Yammer defines engagement
as having made some type of server call by interacting with the product (shown in the data as
events of type “engagement”). Any point in this chart can be interpreted as “the number of users
who logged at least one engagement event during the week starting on that date.”
You are responsible for determining what caused the dip at the end of the chart shown above
and, if appropriate, recommending solutions for the problem.
    

__Hypothesis__:
    
1. Holidays affected this drop
2. App or web or device problem
3. Code is broken for specific event log or specific location/region

__Import all libraries and dependencies__

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import dates
from datetime import datetime
import random
import mysql.connector as mysql
import pandasql as ps
from pandasql import *

In [11]:
#read data

# Input Dataset for the Model

def ReadFromCSV(FilePath, sckipLines = 0, sep = ','):  

    #Read and clean the csv file in FilePath ignoring the first sckipLines lines.

    #(omits non-ascii characters from columns' name)

    xa = pd.read_csv(FilePath, header=sckipLines, sep = sep, low_memory=False, index_col=False)

    return xa

Users Table: includes one row per user, with descriptive information about that user's account

In [12]:
users=ReadFromCSV('/Users/mojganmazouchi/Desktop/Yammer/data/yammer_users.csv') 


print(users.shape)  #(19066, 5)

#df=df.drop(['Index'],axis=1)
users.head()

(19066, 6)


,user_id,created_at,company_id,language,activated_at,state
0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
1,1.0,2013-01-01 13:07:46,28.0,english,NaN,pending
2,2.0,2013-01-01 10:59:05,51.0,english,NaN,pending
3,3.0,2013-01-01 18:40:36,2800.0,german,2013-01-01 18:42:02,active
4,4.0,2013-01-01 14:37:51,5110.0,indian,2013-01-01 14:39:05,active
